In [32]:
import pandas as pd

# import fdf from the previous work 'avgFourDays_3'
fdf = pd.read_csv('fdf.csv')
print("fdf :\n", fdf.head())

fdf :
    출발영업소코드  도착영업소코드 출발영업소명 도착영업소명   도착지방향평균   출발지방향평균
0      101      101     서울     서울     61.75     61.75
1      101      102     서울    동수원   3706.75   5166.50
2      101      103     서울   수원신갈  16726.50  16281.25
3      101      104     서울     지곡     17.75     15.00
4      101      105     서울     기흥   9217.75   9065.50


In [33]:
# sdf stands for station dataFrame
sdf = pd.read_excel('노선별영업소현황.xlsx')
print("sdf :\n", sdf.head())

sdf :
    노선명  영업소명          전화번호                팩스번호  우편번호\n(5자리)  \
0  경부선    오산  031-327-2128        031-894-4128      18150.0   
1  경부선    기흥  031-327-2126        031-894-4126      17086.0   
2  경부선  기흥동탄  031-327-2127        031-894-4127      17102.0   
3  경부선  수원신갈  031-327-2125  031-894-4125(4159)      17096.0   
4  경부선    서울  031-327-2124        031-894-4124      13550.0   

                               주소  
0             경기도 오산시 원동로 105(원동)  
1           경기도 용인시 기흥구 고매동 519-1  
2  경기도 용인시 기흥구 경부고속도로 387번지 (고매동)  
3       경기도 용인시 기흥구 하갈로 240 (상갈동)  
4     경기도 성남시 분당구 대왕판교로 240 (궁내동)  


In [35]:
# feature engineering on sdf['주소'] before it is joined with fdf
# with the use of .split() method. The followings are examples :
print(sdf['주소'][0])
print(sdf['주소'][0].split())
print(sdf['주소'][0].split()[0])
print(sdf['주소'][0].split()[1])

경기도 오산시 원동로 105(원동)
['경기도', '오산시', '원동로', '105(원동)']
경기도
오산시


In [41]:
# 세부주소1 refers to province, while 세부주소2 does to province or municipality
# the latter depends on whether a region concerns 경상남도
sdf['세부주소1'] = sdf['주소'].apply(lambda x: x.split()[0]) 
sdf['세부주소2'] = sdf['주소'].apply(lambda x: x.split()[0] if x.split()[0]!='경상남도' else x.split()[1])

In [42]:
# check the following two cells out :
# as for stations outside 경상남도, both detailed addresses have the same value refering to province
# for example,
bMask = (sdf['영업소명']=='북다사') | (sdf['영업소명']=='양양')
sdf[bMask]

,노선명,영업소명,전화번호,팩스번호,우편번호\n(5자리),주소,세부주소1,세부주소2
122,대구외곽순환선,북다사,053-717-2786,053-715-4786,42907.0,대구광역시 달성군 다사읍 다사로 320,대구광역시,대구광역시
142,동해선,양양,033-812-2228,033-810-4228,25035.0,강원도 양양군 곰밭길 175-32,강원도,강원도


In [40]:
# on contrary, 세부주소2 refers to a certain municipality when it comes to stations inside 경상남도
bMask = (sdf['영업소명']=='양산') | (sdf['영업소명']=='장유')
sdf[bMask]

,노선명,영업소명,전화번호,팩스번호,우편번호\n(5자리),주소,세부주소1,세부주소2
34,경부선,양산,055-711-2822,055-717-4822,50565.0,경상남도 양산시 상북면 와곡2길 12-1,경상남도,양산시
63,남해 제2지선,장유,055-711-2811,055-717-4811,50983.0,경상남도 김해시 장유로 324번길 44-43,경상남도,김해시


In [43]:
# for the sake of simplicity, drop redundant columns
# 노선명 won't be directly used, either, but let's just leave it there for reference
sdf = sdf.drop(columns=['전화번호','팩스번호','우편번호\n(5자리)','주소'])
sdf.head()

,노선명,영업소명,세부주소1,세부주소2
0,경부선,오산,경기도,경기도
1,경부선,기흥,경기도,경기도
2,경부선,기흥동탄,경기도,경기도
3,경부선,수원신갈,경기도,경기도
4,경부선,서울,경기도,경기도


In [55]:
# before join the two fdf and sdf on 영업소명, check whether the two 영업소명 are compatible
fdfStation1 = sorted(fdf['출발영업소명'].unique())
fdfStation2 = sorted(fdf['도착영업소명'].unique())
sdfStation = sorted(sdf['영업소명'].unique())

# check the number of their elements
print(len(fdfStation1))
print(len(fdfStation2))
print(len(sdfStation))

# well they do not match well !

429
429
370


In [56]:
# construct a dataFrame with them and convert it to csv
compStation = pd.DataFrame({'fdf':pd.Series(fdfStation1), 'sdf':pd.Series(sdfStation)})
compStation.to_csv('compStation.csv', index=False)

In [ ]:
# the data list is quite long!
# better filter out lines passing through 경상남도


In [3]:
myNode = ['거제시','거창군','고성군','김해시','남해군','밀양시','사천시','산청군','양산시','의령군','진주시','창녕군'
         ,'창원시','통영시','하동군','함안군','함양군','합천군','경상북도','전라북도','전라남도','울산광역시','부산광역시']
len(myNode)

23

In [7]:
import numpy as np
adjMatrix = pd.DataFrame(np.zeros((23,23)), columns=myNode, index=myNode)
adjMatrix.head()

,거제시,거창군,고성군,김해시,남해군,밀양시,사천시,산청군,양산시,의령군,...,통영시,하동군,함안군,함양군,합천군,경상북도,전라북도,전라남도,울산광역시,부산광역시
거제시,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
거창군,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
고성군,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
김해시,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
남해군,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
# use the code from the previous work 'checkRoute_2' for reference
# when it comes to 경부선(부산->양산)
## check the figures again before adding them to the adjMatrix 
startCode = 529
destCode = 138
bMask1 = (df['출발영업소코드']==startCode) & (df['도착영업소코드']==destCode)
bMask2 = (df['출발영업소코드']==destCode) & (df['도착영업소코드']==startCode)
print(df[bMask1])
print(df[bMask2])

       출발영업소코드  도착영업소코드 출발영업소명 도착영업소명  도착지방향평균  출발지방향평균
97420      529      138     노포     양산    306.0    405.5
       출발영업소코드  도착영업소코드 출발영업소명 도착영업소명  도착지방향평균  출발지방향평균
16100      138      529     양산     노포    405.5    306.0


In [ ]:
# fill up adjMatrix with data from df
# forward
adjMatrix['부산광역시']['양산시'] += df[bMask1]['도착지방향평균']
# backward
adjMatrix['양산시']['부산광역시'] += df[bMask1]['도착지방향평균']